In [78]:
import os
import pandas as pd

func_list = ['mean', 'max']

  # aws 지역명 매핑용 데이터
new_AWS = pd.read_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/new_AWS_final.csv',encoding='cp949')

  # 결측치 확인
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

  # t4~t8 컬럼 삭제
def drop_col_t4_t8(df):
  drop_col = ['t4',	't5',	't6',	't7',	't8']
  if drop_col[0] in df.columns:
    df.drop(drop_col, axis=1, inplace=True)
  return df

  # area 매핑
def mapping_aws(df):
    df = pd.merge(df, new_AWS, on='stn_id', how='left')
    df.drop(['stn_id','지점명(한글)'], axis=1, inplace=True)
    return df

  # 발표시각(tm_fc) 관측시간 오전 6시만 살림(관측 년,월,일은 살림)
def remove_hour(df):
  df = df.loc[df['tm_fc'].str.contains('06:00:00')]
  df['tm_fc'] = df.tm_fc.str.split().str[0]
  return df

  # 지역명, 발표시간 기준으로 그룹화(최고값, 평균값)
def groupby_tm_area(df,start,end,group_key):
  df.fillna({'t1':0,'t2':0,'t3':0},inplace=True)
  cols = df.columns[start:end].tolist()
  df = df.groupby(group_key)[cols].agg(func_list).reset_index()
  df.columns = list(map('_'.join, df.columns.values))
  df = df.rename(columns={'tm_fc_' : 'tm_fc','area_' : 'area'})
  return df

  # 예측 column 옆에 코드 각인 ( ex) A03_t2_max )
def engrave_code(df, cod):
  change_names = ['t1_mean','t1_max','t2_mean','t2_max','t3_mean','t3_max']
  for name in change_names:
    df.rename(columns={str(name):str(cod)+'_'+str(name)},inplace=True)
  return df

  # 발표시각(tm_fc) 형변환
def cast_tm_fc(df):
  df.tm_fc =pd.to_datetime(df.tm_fc)
  df.tm_fc = df.tm_fc.dt.strftime('%Y%m%d')
  return df

  #t2_,t3_ 의 값을 각각 아래로 1칸,2칸 씩 이동
def match_forecast_data(df,start,end):
  if len(df) > 2:
    mid = (start+end)//2
    df.iloc[:,start:mid] = df.iloc[:,start:mid].shift(1)
    df.iloc[:,mid:end] = df.iloc[:,mid:end].shift(2)
  return df


  # 정렬 후 아래 값으로 결측치 채우기
def sort_fill(df, sort_key):
  df.sort_values(by=sort_key,inplace=True,na_position='last',ignore_index=True)
  df.fillna(method= 'bfill',inplace=True)
  return df

  # fct 데이터 지역명 변경
def area_words(word):
    if (str(type(word))!="<class 'str'>"):
      return word

    if word in ['경상북도', '충청북도', '충청남도', '경상남도', '전라북도', '전라남도']:
        word = list(word)[0] + list(word)[2]
    elif word == '이어도':
        word = '제주'
    else:
        word = list(word)[0] + list(word)[1]
    return word 


# aws 불러오기 및 전처리
path = '/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/fct_life/mapping_aws/'
file_list = os.listdir(path)

file_name_comm = 'fct_life_'
year_for_A03 = year_for_A04 = year_for_A05 = year_for_A06 = year_for_A07 = 0

data_aws_A03 = []
data_aws_A04 = []
data_aws_A05 = []
data_aws_A06 = []
data_aws_A07 = []

res_aws_data = [pd.DataFrame() for _ in range(5)]

for i in file_list:
  if 'A03' in i:
    target = 'A03'
    res = pd.read_csv(path + file_name_comm+str(2012+year_for_A03)+'_'+target+'_aws.csv',encoding='cp949',index_col=0)
    data_aws_A03.append(res)
    res_aws_data[0] = pd.concat(data_aws_A03, axis=0, ignore_index = True)
    year_for_A03+=1
  
  elif 'A04' in i:
    target = 'A04'
    res = pd.read_csv(path + file_name_comm+str(2012+year_for_A04)+'_'+target+'_aws.csv',encoding='cp949',index_col=0)
    data_aws_A04.append(res)
    res_aws_data[1] = pd.concat(data_aws_A04, axis=0, ignore_index = True)
    year_for_A04+=1

  elif 'A05' in i:
    target = 'A05'
    res = pd.read_csv(path + file_name_comm+str(2012+year_for_A05)+'_'+target+'_aws.csv',encoding='cp949',index_col=0)
    data_aws_A05.append(res)
    res_aws_data[2] = pd.concat(data_aws_A05, axis=0, ignore_index = True)
    year_for_A05+=1

  elif 'A06' in i:
    target = 'A06'
    res = pd.read_csv(path + file_name_comm+str(2012+year_for_A06)+'_'+target+'_aws.csv',encoding='cp949',index_col=0)
    data_aws_A06.append(res)
    res_aws_data[3] = pd.concat(data_aws_A06, axis=0, ignore_index = True)
    year_for_A06+=1

  elif 'A07' in i:
    target = 'A07'
    res = pd.read_csv(path + file_name_comm+str(2012+year_for_A07)+'_'+target+'_aws.csv',encoding='cp949',index_col=0)
    data_aws_A07.append(res)
    res_aws_data[4] = pd.concat(data_aws_A07, axis=0, ignore_index = True)
    year_for_A07+=1

for i in range(len(res_aws_data)):
  res_aws_data[i] =drop_col_t4_t8(res_aws_data[i].copy())
  res_aws_data[i] = mapping_aws(res_aws_data[i].copy())
  res_aws_data[i] = remove_hour(res_aws_data[i].copy())
  res_aws_data[i] = groupby_tm_area(res_aws_data[i].copy(),2,5,['tm_fc','area'])
  code = 'A0'+str(i+3)
  res_aws_data[i] = engrave_code(res_aws_data[i].copy(),code)
  res_aws_data[i] = cast_tm_fc(res_aws_data[i].copy())


# fct 불러오기 및 전처리
def fct_preprocess(df,code):
  df=drop_col_t4_t8(df)
  df=remove_hour(df.copy())
  df = df.rename(columns={'1단계' : 'area'})
  df['area'] = [area_words(i) for i in df['area']]
  df.dropna(subset=['area'])
  df=groupby_tm_area(df,3,6,['tm_fc','area'])
  df=engrave_code(df, code)
  df = cast_tm_fc(df)
  
  return df

path = '/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/fct_life/mapping_fct/'

res_fct_data = [pd.DataFrame() for _ in range(5)]
data_list = []

# A03
fct_A03_2012 = pd.read_csv(path+'fct_life_2012_A03_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A03_2012 = fct_preprocess(fct_A03_2012,'A03')
data_list.append(fct_A03_2012)
fct_A03_2013 = pd.read_csv(path+'fct_life_2013_A03_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A03_2013 =fct_preprocess(fct_A03_2013,'A03')
data_list.append(fct_A03_2013)
fct_A03_2014 = pd.read_csv(path+'fct_life_2014_A03_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A03_2014 =fct_preprocess(fct_A03_2014,'A03')
data_list.append(fct_A03_2014)
fct_A03_2015 = pd.read_csv(path+'fct_life_2015_A03_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A03_2015 = fct_preprocess(fct_A03_2015,'A03')
data_list.append(fct_A03_2015)
fct_A03_2016 = pd.read_csv(path+'fct_life_2016_A03_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A03_2016 = fct_preprocess(fct_A03_2016,'A03')
data_list.append(fct_A03_2016)
res_fct_data[0] = pd.concat(data_list, axis=0, ignore_index = True)
del fct_A03_2012,fct_A03_2013,fct_A03_2014,fct_A03_2015,fct_A03_2016

data_list = []

#A04
fct_A04_2012 = pd.read_csv(path+'fct_life_2012_A04_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A04_2012 = fct_preprocess(fct_A04_2012,'A04')
data_list.append(fct_A04_2012)
fct_A04_2013 = pd.read_csv(path+'fct_life_2013_A04_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A04_2013 = fct_preprocess(fct_A04_2013,'A04')
data_list.append(fct_A04_2013)
fct_A04_2014 = pd.read_csv(path+'fct_life_2014_A04_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A04_2014 = fct_preprocess(fct_A04_2014,'A04')
data_list.append(fct_A04_2014)
fct_A04_2015 = pd.read_csv(path+'fct_life_2015_A04_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A04_2015 = fct_preprocess(fct_A04_2015,'A04')
data_list.append(fct_A04_2015)
fct_A04_2016 = pd.read_csv(path+'fct_life_2016_A04_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A04_2016 = fct_preprocess(fct_A04_2016,'A04')
data_list.append(fct_A04_2016)
res_fct_data[1] = pd.concat(data_list, axis=0, ignore_index = True)
del fct_A04_2012,fct_A04_2013,fct_A04_2014,fct_A04_2015,fct_A04_2016

data_list = []

#A05
fct_A05_2012 = pd.read_csv(path+'fct_life_2012_A05_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A05_2012 = fct_preprocess(fct_A05_2012,'A05')
data_list.append(fct_A05_2012)
fct_A05_2013 = pd.read_csv(path+'fct_life_2013_A05_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A05_2013 = fct_preprocess(fct_A05_2013,'A05')
data_list.append(fct_A05_2013)
fct_A05_2014 = pd.read_csv(path+'fct_life_2014_A05_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A05_2014 = fct_preprocess(fct_A05_2014,'A05')
data_list.append(fct_A05_2014)
fct_A05_2015 = pd.read_csv(path+'fct_life_2015_A05_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A05_2015 = fct_preprocess(fct_A05_2015,'A05')
data_list.append(fct_A05_2015)
fct_A05_2016 = pd.read_csv(path+'fct_life_2016_A05_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A05_2016 = fct_preprocess(fct_A05_2016,'A05')
data_list.append(fct_A05_2016)
res_fct_data[2] = pd.concat(data_list, axis=0, ignore_index = True)
del fct_A05_2012,fct_A05_2013,fct_A05_2014,fct_A05_2015,fct_A05_2016

data_list = []

#A06
fct_A06_2012 = pd.read_csv(path+'fct_life_2012_A06_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A06_2012 = fct_preprocess(fct_A06_2012,'A06')
data_list.append(fct_A06_2012)
fct_A06_2013 = pd.read_csv(path+'fct_life_2013_A06_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A06_2013 = fct_preprocess(fct_A06_2013,'A06')
data_list.append(fct_A06_2013)
fct_A06_2014 = pd.read_csv(path+'fct_life_2014_A06_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A06_2014 = fct_preprocess(fct_A06_2014,'A06')
data_list.append(fct_A06_2014)
fct_A06_2015 = pd.read_csv(path+'fct_life_2015_A06_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A06_2015 = fct_preprocess(fct_A06_2015,'A06')
data_list.append(fct_A06_2015)
fct_A06_2016 = pd.read_csv(path+'fct_life_2016_A06_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A06_2016 = fct_preprocess(fct_A06_2016,'A06')
data_list.append(fct_A06_2016)
res_fct_data[3] = pd.concat(data_list, axis=0, ignore_index = True)
del fct_A06_2012,fct_A06_2013,fct_A06_2014,fct_A06_2015,fct_A06_2016

data_list = []

#A07
fct_A07_2012 = pd.read_csv(path+'fct_life_2012_A07_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A07_2012 = fct_preprocess(fct_A07_2012,'A07')
data_list.append(fct_A07_2012)
fct_A07_2013 = pd.read_csv(path+'fct_life_2013_A07_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A07_2013 = fct_preprocess(fct_A07_2013,'A07')
data_list.append(fct_A07_2013)
fct_A07_2014 = pd.read_csv(path+'fct_life_2014_A07_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A07_2014 = fct_preprocess(fct_A07_2014,'A07')
data_list.append(fct_A07_2014)
fct_A07_2015 = pd.read_csv(path+'fct_life_2015_A07_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A07_2015 = fct_preprocess(fct_A07_2015,'A07')
data_list.append(fct_A07_2015)
fct_A07_2016 = pd.read_csv(path+'fct_life_2016_A07_fct.csv',encoding='cp949',index_col=0,error_bad_lines=False)
fct_A07_2016 = fct_preprocess(fct_A07_2016,'A07')
data_list.append(fct_A07_2016)
res_fct_data[4] = pd.concat(data_list, axis=0, ignore_index = True)
del fct_A07_2012,fct_A07_2013,fct_A07_2014,fct_A07_2015,fct_A07_2016

del data_list


# aws , fct 병합
res_data = [pd.DataFrame() for _ in range(5)]

for i in range(5):
  new_col_name = 'A0'+str(i+3)

  # aws와 fct의 tm_fc와 area의 교집합을 구하고, t1_,t2_,t3_ 의 평균값을 구한다.
  intersected_df = pd.merge(res_aws_data[i], res_fct_data[i], on=['tm_fc','area'], how='inner')
  intersected_df[new_col_name+'_t1_mean']=intersected_df.iloc[:,[2,8]].mean(axis=1)
  intersected_df[new_col_name+'_t1_max']=intersected_df.iloc[:,[3,9]].mean(axis=1)
  intersected_df[new_col_name+'_t2_mean']=intersected_df.iloc[:,[4,10]].mean(axis=1)
  intersected_df[new_col_name+'_t2_max']=intersected_df.iloc[:,[5,11]].mean(axis=1)
  intersected_df[new_col_name+'_t3_mean']=intersected_df.iloc[:,[6,12]].mean(axis=1)
  intersected_df[new_col_name+'_t3_max']=intersected_df.iloc[:,[7,13]].mean(axis=1)
  intersected_df.drop(intersected_df.columns[2:14],axis = 1,inplace = True)
  # 차집합(aws - fct) 구하고, t1_,t2_,t3_을 유지한다.
  set_diff_df_1 = pd.concat([res_aws_data[i], res_fct_data[i], res_fct_data[i]]).drop_duplicates(subset=['tm_fc', 'area'],keep=False)
  # 차집합(fct - aws) 구하고, t1_,t2_,t3_을 유지한다.
  set_diff_df_2 = pd.concat([res_fct_data[i], res_aws_data[i], res_aws_data[i]]).drop_duplicates(subset=['tm_fc', 'area'],keep=False)


  # intersected_df, set_diff_df_1, set_diff_df_2 병합
  res_data[i] = pd.concat([intersected_df, set_diff_df_1, set_diff_df_2],axis=0,ignore_index=True)

# res_data 전체 outer join
res = pd.DataFrame(columns=['tm_fc','area'])
for i in range(5):
  res = pd.merge(res,res_data[i],on=['tm_fc',	'area'],how='outer')
res.sort_values(by=['tm_fc','area'],inplace=True)
res = res.reset_index(drop=True)
res.rename(columns={'tm_fc':'yyyymmdd'},inplace=True)
res = res.astype({'yyyymmdd':int})

final = pd.read_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/final/final.csv',encoding='cp949')

final_res = pd.merge(final,res,on=['yyyymmdd','area'],how='outer')

# 결측치 보정
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

lr = LinearRegression()

  # 새로 추가된 데이터에 결측값을 99998789(안쓰일거 같은 값)로 채움
final_cpy = final_res.copy()
final_cpy = final_cpy.iloc[:,71:101]
final_cpy.fillna(99998789,inplace=True)
final_res.iloc[:,71:101] = final_cpy

  # 성별 까지 학습시키기 위해서 칼럼 순서를 바꿈 (~ sex frequency ~  => ~ frequency sex ~)
col_1 = final_res.columns[:2].to_list()
col_2 = final_res.columns[4:].to_list()
new_col=col_1+['frequency','sex']+col_2
final_res=final_res[new_col]

imp = IterativeImputer(estimator=lr,missing_values=99998789, verbose=0, min_value = 0, imputation_order='roman',random_state=42)
w1 = final_res.drop(['yyyymmdd', 'area','frequency'], axis=1)
w2 = final_res[['yyyymmdd', 'area','frequency']]
X = imp.fit_transform(w1)
final_res = pd.concat([w2,pd.DataFrame(X, columns = final_res.columns[3:])], axis=1)

  # 칼럼 순서 원상태로 복구 (~ frequency sex ~  => ~ sex frequency ~)
col_1 = final_res.columns[:2].to_list()
col_2 = final_res.columns[4:].to_list()
new_col=col_1+['sex','frequency']+col_2
final_res=final_res[new_col]



# # 지연효과 반영 (t2,t3 각각 1칸 2칸 씩 내림)
# area_names = final_res['area'].unique()

# alt_df = pd.DataFrame()
# for area in area_names:
#   for sex in [1,2]:
#     tmp_df = final_res[(final_res['area'] == area)&(final_res['sex'] == sex)]
#     tmp_df = match_forecast_data(tmp_df,73,77)
#     tmp_df = match_forecast_data(tmp_df,79,83)
#     tmp_df = match_forecast_data(tmp_df,85,89)
#     tmp_df = match_forecast_data(tmp_df,91,95)
#     tmp_df = match_forecast_data(tmp_df,97,101)
#     sort_key = ['yyyymmdd','area','sex']
#     tmp_df = sort_fill(tmp_df, sort_key)
#     alt_df = pd.concat([alt_df,tmp_df],axis=0)
    
# final_res = alt_df
final_res.sort_values(['yyyymmdd','area','sex'],inplace=True)
final_res.reset_index(drop=True,inplace=True)

final_res.drop(['tot_person','num_risk_age','year', 'month', 'day', 'day_differ', 'month_differ', 'avg_tca_mean', 'avg_tca_max',
 'sum_ss_hr_mean', 'sum_ss_hr_max', 'ssrate_mean', 'ssrate_max', 'avg_rhm_mean', 'avg_rhm_min', 'avg_rhm_max', 'min_rhm_mean', 'min_rhm_min', 'min_rhm_max', 'sum_rn_mean', 'sum_rn_min',
 'sum_rn_max', 'mi10_max_rn_mean', 'mi10_max_rn_min', 'mi10_max_rn_max', 'hr1_max_rn_mean', 'hr1_max_rn_min', 'hr1_max_rn_max', 
 'hr6_max_rn_mean', 'hr6_max_rn_min', 'hr6_max_rn_max', 'avg_ws_mean', 'avg_ws_min', 'avg_ws_max', 'max_ws_mean', 'max_ws_min', 'max_ws_max',
 'avg_min_tg_mean', 'avg_min_tg_min', 'avg_min_tg_max', 'min_tg_mean', 'min_tg_min', 'min_tg_max', 'avg_ta_mean', 'avg_ta_min', 'avg_ta_max', 
 'max_ta_mean', 'max_ta_min', 'max_ta_max', 'min_ta_mean', 'min_ta_min', 'min_ta_max', 'SO2_mean', 'SO2_min', 'SO2_max','CO_mean','CO_min',
 'CO_max','O3_mean','O3_min','O3_max','NO2_mean','NO2_min','NO2_max','PM10_mean','PM10_min','PM10_max','A07_t3_mean','A07_t3_max'],axis = 1 , inplace=True)

# 최종데이터
final_res.to_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/final/final_res.csv',encoding='cp949',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
